#### combining speech recognition, response generation, emotion recognition models
#### talk --> speech to text --> create response --> detect emotion --> send data to unity

imports

In [1]:
from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np

import time, logging
from datetime import datetime
import collections, queue, os, os.path
import deepspeech
import pyaudio
import wave
import webrtcvad
from halo import Halo
from scipy import signal

import itertools
import threading
from threading import Thread

import speech_recognition as sr
import pyaudio

import pandas as pd
import numpy as np
import ktrain
from ktrain import text

import random
import sched

import sys
sys.path.append("C:/Users/Munia/Documents/Anaconda/Master Thesis Project/vh-models/text to speech/TensorFlowTTS-master")
import soundfile as sf
import yaml
import tensorflow as tf
from tensorflow_tts.inference.auto_model import TFAutoModel
from tensorflow_tts.inference.auto_processor import AutoProcessor

import server # the server.py script for udp connection
import deepspeech_mic_vad_streaming # the mic_vad_streaming script for deepspeech
import argparse

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Munia\.conda\envs\VHP\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-58b91fb9348a>", line 1, in <module>
    from keras.preprocessing.image import img_to_array
  File "C:\Users\Munia\.conda\envs\VHP\lib\site-packages\keras\__init__.py", line 3, in <module>
    from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
  File "C:\Users\Munia\.conda\envs\VHP\lib\site-packages\tensorflow\__init__.py", line 41, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "C:\Users\Munia\.conda\envs\VHP\lib\site-packages\tensorflow\python\__init__.py", line 45, in <module>
    from tensorflow.python import data
  File "C:\Users\Munia\.conda\envs\VHP\lib\site-packages\tensorflow\python\data\__init__.py", line 25, in <module>
    from tensorflow.python.data import experimental
  File "C:\Use

TypeError: object of type 'NoneType' has no len()

#### load models

In [ ]:
main_path = 'C:/Users/Munia/Documents/Anaconda/Master Thesis Project/vh-models/'


# load text based emotion recognition model
t_predictor_path = main_path + "emotion-recognition/emotion recognition from text/Bert/models/bert_model/"
t_predictor = ktrain.load_predictor(t_predictor_path)

# load video based emotion recognition models
v_detection_model_path = main_path + 'emotion-recognition/real time emotion detection/haarcascade_files/haarcascade_frontalface_default.xml'
v_emotion_model_path = main_path + 'emotion-recognition/real time emotion detection/models/XCEPTION-102-0.66.hdf5'
face_detection = cv2.CascadeClassifier(v_detection_model_path)
v_emotion_classifier = load_model(v_emotion_model_path, compile=False)
chatbot_model = 'open-domain-chatbot/models/blender/blender_90M/model'

### speech to text method 1 deepspeech

In [ ]:
# function for recognizing speech with deepspeech
def speak_1():
    DEFAULT_SAMPLE_RATE = 16000
    parser = argparse.ArgumentParser(description="Stream from microphone to DeepSpeech using VAD")
    parser.add_argument('-v', '--vad_aggressiveness', type=int, default=3,
                        help="Set aggressiveness of VAD: an integer between 0 and 3, 0 being the least aggressive about filtering out non-speech, 3 the most aggressive. Default: 3")
    parser.add_argument('--nospinner', action='store_true',
                        help="Disable spinner")
    parser.add_argument('-w', '--savewav',
                        help="Save .wav files of utterences to given directory")
    parser.add_argument('-f', '--file',
                        help="Read from .wav file instead of microphone")
    #parser.add_argument('-m', '--model', required=True,
    #                    help="Path to the model (protocol buffer binary file, or entire directory containing all standard-named files for model)")
    #parser.add_argument('-s', '--scorer',
    #                    help="Path to the external scorer file.")
    parser.add_argument('-d', '--device', type=int, default=None,
                        help="Device input index (Int) as listed by pyaudio.PyAudio.get_device_info_by_index(). If not provided, falls back to PyAudio.get_default_device().")
    parser.add_argument('-r', '--rate', type=int, default=DEFAULT_SAMPLE_RATE,
                        help=f"Input device sample rate. Default: {DEFAULT_SAMPLE_RATE}. Your device may require 44100.")
    ARGS = parser.parse_args()
    if ARGS.savewav: os.makedirs(ARGS.savewav, exist_ok=True)

    deepspeech_mic_vad_streaming.main(ARGS)

### speech to text method 2 Google's SpeechRecognition

In [ ]:
# function for loading animation while receiving speech
# animation will be for x seconds
def loading():
    done = False
    #here is the animation
    def animate():
        for c in itertools.cycle(['|', '/', '-', '\\']):
            if done:
                break
            sys.stdout.write('\rloading ' + c)
            sys.stdout.flush()
            time.sleep(0.1)
        sys.stdout.write('\rDone!     ')

    t = threading.Thread(target=animate)
    t.start()

    #long process here
    time.sleep(5)
    done = True
    

# function for recognizing speech
# speech recognition stops after x seconds
def speak_2():
    # obtain audio from the microphone
    global sentence
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something!")
        #loading()
        # stop speaking after 5 seconds
        audio = recognizer.listen(source, timeout=5)
        print("processing..")
    try:
        sentence = recognizer.recognize_google(audio)
        text_file = open("outputs/text-output.txt", "w")
        #write string to file
        text_file.write(sentence)
        #close file
        text_file.close()
        print("***DONE***\n\n")
    except sr.UnknownValueError:
        print("\nGoogle Speech Recognition could not understand audio")
        speak()
    except sr.RequestError as e:
        print("\nCould not request results from Google Speech Recognition service; {0}".format(e))

### text to speech

In [ ]:
def synthesize(responsetxt):
    # initialize fastspeech2 model.
    fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")

    # initialize mb_melgan model
    mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")

    # inference
    processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")

    input_ids = processor.text_to_sequence(responsetxt)
    # fastspeech inference
    
    mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
    input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
    speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
    speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
    energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
    )

    # melgan inference
    audio_before = mb_melgan.inference(mel_before)[0, :, 0]
    audio_after = mb_melgan.inference(mel_after)[0, :, 0]

    # save to file
    folder = "F:/Unity/MASTER THESIS/VHProject-master/Assets/Resources"
    #sf.write(folder+'/audio_before.wav', audio_before, 22050, "PCM_16")
    sf.write(folder+'/demo.wav', audio_after, 22050, "PCM_16")

### emotion recognition

#### from text

In [ ]:
def text_based_emotion(text):
    global t_emotion
    global t_prob
    EMOTIONS = ['happy', 'sad', 'scared', 'angry', 'neutral']
    
    start_time = time.time() 
    preds = t_predictor.predict(text, return_proba=True)
    emotion_probability = np.max(preds)
    emotion_probability = str(round(emotion_probability, 2))
    #convert probability from string to float
    probability = float(emotion_probability)
    #print("blah blah blah: ", probability)
    label = EMOTIONS[preds.argmax()]

    #print('predicted: {} {} ({:.2f})'.format(label, emotion_probability, (time.time() - start_time)))
    t_emotion = label
    t_prob = probability
    
    #return label, emotion_probability

In [ ]:
# video capture is set to play for 5 seconds
def video_based_emotion():
    
    #main_path = 'C:/Users/Munia/Documents/Anaconda/vh-models/emotion-recognition/real time emotion detection/'
    global v_emotion
    global v_prob
    
    EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised", "neutral"]

    cv2.namedWindow('your_face')
    t0 = time.time() # start time in seconds
    camera = cv2.VideoCapture(0)
    while True:
        frame = camera.read()[1]
        #reading the frame
        frame = imutils.resize(frame,width=500)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)

        canvas = np.zeros((250, 300, 3), dtype="uint8")
        frameClone = frame.copy()
        if len(faces) > 0:
            faces = sorted(faces, reverse=True,
            key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
            (fX, fY, fW, fH) = faces

            roi = gray[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (64, 64))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)


            preds = v_emotion_classifier.predict(roi)[0]
            #print(preds)
            emotion_probability = np.max(preds)
            #print(np.max(preds))
            label = EMOTIONS[preds.argmax()]
            #print(label)
        else: continue

        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                    # construct the label text
                    text = "{}: {:.2f}%".format(emotion, prob * 100)

                    w = int(prob * 300)
                    cv2.rectangle(canvas, (7, (i * 35) + 5),
                    (w, (i * 35) + 35), (0, 0, 255), -1)
                    cv2.putText(canvas, text, (10, (i * 35) + 23),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                    (255, 255, 255), 2)
                    cv2.putText(frameClone, label, (fX, fY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                    cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                                  (0, 0, 255), 2)
        t1 = time.time() # current time
        num_seconds = t1 - t0 # diff

        cv2.imshow('your_face', frameClone)
        cv2.imshow("Probabilities", canvas)
        
        # stop video capture after x seconds
        if num_seconds > 5:  # e.g. break after 30 seconds
            break
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break
    
    camera.release()
    cv2.destroyAllWindows()
    v_emotion = label
    v_prob = emotion_probability
    

In [ ]:
# last function for emotion detection that comapres between emotion from video and emotion from text
def predict_emotion():
    emotionsList = ["happy", "sad", "neutral", "angry", "scared", "surprised", "disgust"]
    final_emotion = ""
    final_prob = 0
    
    if (t_emotion==v_emotion):
        final_emotion = t_emotion
        final_prob = t_prob
        return final_emotion, final_prob
    else:
        MAX = max(t_prob, v_prob) 
        if(MAX == t_prob):
            final_emotion = t_emotion
            final_prob = t_prob
            return final_emotion, final_prob
        elif(MAX == v_prob):
            final_emotion = v_emotion
            final_prob = v_prob
            return final_emotion, final_prob

### generate response
### response generation with ParlAI Blender 90M Model

In [ ]:
""""
LocalHumanAgent Class (for keyboard input) to let human reply replacing an ML agent
instead of importing  LocalHumanAgent.py Script

""""

In [ ]:
"""
Agent does gets the local keyboard input in the act() function.
Example: parlai eval_model -m local_human -t babi:Task1k:1 -dt valid
"""

from typing import Optional
from parlai.core.params import ParlaiParser
from parlai.core.opt import Opt
from parlai.core.agents import Agent
from parlai.core.message import Message
from parlai.utils.misc import display_messages, load_cands
from parlai.utils.strings import colorize


class LocalHumanAgent(Agent):
    @classmethod
    def add_cmdline_args(
        cls, parser: ParlaiParser, partial_opt: Optional[Opt] = None
    ) -> ParlaiParser:
        """
        Add command-line arguments specifically for this agent.
        """
        agent = parser.add_argument_group('Local Human Arguments')
        agent.add_argument(
            '-fixedCands',
            '--local-human-candidates-file',
            default=None,
            type=str,
            help='File of label_candidates to send to other agent',
        )
        agent.add_argument(
            '--single_turn',
            type='bool',
            default=False,
            help='If on, assumes single turn episodes.',
        )
        return parser

    def __init__(self, opt, shared=None):
        super().__init__(opt)
        self.id = 'localHuman'
        self.episodeDone = False
        self.finished = False
        self.fixedCands_txt = load_cands(self.opt.get('local_human_candidates_file'))
        print(
            colorize(
                "Enter [DONE] if you want to end the episode, [EXIT] to quit.",
                'highlight',
            )
        )

    def epoch_done(self):
        return self.finished

    def observe(self, msg):
        print(
            display_messages(
                [msg],
                add_fields=self.opt.get('display_add_fields', ''),
                prettify=self.opt.get('display_prettify', False),
                verbose=self.opt.get('verbose', False),
            )
        )

    def act(self):
        reply = Message()
        reply['id'] = self.getID()
        try:
            ## input message from user
            #reply_text = input(colorize("Enter Your Message:", 'text') + ' ')
            reply_text = sentence
        except EOFError:
            self.finished = True
            return {'episode_done': True}

        reply_text = reply_text.replace('\\n', '\n')
        reply['episode_done'] = False
        if self.opt.get('single_turn', False):
            reply.force_set('episode_done', True)
        reply['label_candidates'] = self.fixedCands_txt
        if '[DONE]' in reply_text:
            # let interactive know we're resetting
            raise StopIteration
        reply['text'] = reply_text
        print("reply['text'] is = ", reply['text'])
        print("reply is = ", reply)
        if '[EXIT]' in reply_text:
            self.finished = True
            raise StopIteration
        return reply

    def episode_done(self):
        return self.episodeDone

In [ ]:
"""
Basic script which allows local human keyboard input to talk to a trained model.

Model = Blender 90M
"""
from parlai.core.params import ParlaiParser
from parlai.core.agents import create_agent
from parlai.core.worlds import create_task
from parlai.core.script import ParlaiScript, register_script
from parlai.utils.world_logging import WorldLogger
import parlai.utils.logging as logging

import random


def setup_args(parser=None):
    if parser is None:
        parser = ParlaiParser(
            True, True, 'Interactive chat with a model on the command line'
        )
    parser.add_argument('-d', '--display-examples', type='bool', default=False)
    parser.add_argument(
        '--display-prettify',
        type='bool',
        default=False,
        help='Set to use a prettytable when displaying '
        'examples with text candidates',
    )
    parser.add_argument(
        '--display-add-fields',
        type=str,
        default='',
        help='Display these fields when verbose is off (e.g., "--display-add-fields label_candidates,beam_texts")',
    )
    parser.add_argument(
        '-it',
        '--interactive-task',
        type='bool',
        default=True,
        help='Create interactive version of task',
    )
    parser.add_argument(
        '--outfile',
        type=str,
        default='',
        help='Saves a jsonl file containing all of the task examples and '
        'model replies. Set to the empty string to not save at all',
    )
    parser.add_argument(
        '--save-format',
        type=str,
        default='conversations',
        choices=['conversations', 'parlai'],
        help='Format to save logs in. conversations is a jsonl format, parlai is a text format.',
    )
    parser.set_defaults(interactive_mode=True, task='interactive')
    LocalHumanAgent.add_cmdline_args(parser, partial_opt=None)
    WorldLogger.add_cmdline_args(parser, partial_opt=None)
    return parser


def interactive(opt):
    global response
    if isinstance(opt, ParlaiParser):
        logging.error('interactive should be passed opt not Parser')
        opt = opt.parse_args()

    # Create model and assign it to the specified task
    agent = create_agent(opt, requireModelExists=True)
    #agent.opt.log()
    human_agent = LocalHumanAgent(opt)
    # set up world logger
    world_logger = WorldLogger(opt) if opt.get('outfile') else None
    world = create_task(opt, [human_agent, agent])

    # Show some example dialogs:
    world.parley()
    #world.get_acts()
    response = world.get_acts()[1]['text']
    #print(world.get_acts()[1]['text'])
    #a = world.get_acts()

@register_script('interactive', aliases=['i'])
class Interactive(ParlaiScript):
    @classmethod
    def setup_args(cls):
        return setup_args()

    def run(self):
        return interactive(self.opt)


In [ ]:
def responseGenerate(msg):
    random.seed(42)
    Interactive.main(model_file=chatbot_model)

### Main Code

#### run all models

In [ ]:
def  print_dialog():
    print("\n\n____________ Dialog ____________")
    print("Me: ", sentence)
    print("emotion from video is: ", v_emotion)
    print("probability  is: ", v_prob)

    print("Bot: ", response)
    print("emotion from text is: ", t_emotion)
    print("probability is: ", t_prob)

    print("----------------------------------------")
    print("Outputs: ")
    print("final emotion output is: ", final_emo, "with probability: ", final_prob)
    print("bot response output is: ", response)


In [ ]:
# first way

In [ ]:
def pipeline():

    while(True):
        t1 = Thread(target = speak_1)
        t2 = Thread(target = video_based_emotion)
        
        t1.start()
        t2.start()
        t1.join()
        t2.join()
        
        output_file = open(main_path + "outputs/text-output.txt", "r")
        sentence = output_file.read()
        
        if(sentence!=None):
            
            output_file = open(main_path + "outputs/text-output.txt", "r")
            sentence = output_file.read()
            output_file.close()
    
            response = responseGenerate(sentence)
            text_based_emotion(response)
            
            break;
            
    final_emo, final_prob = predict_emotion()
    synthesize(response)
    
    if(final_emo!=None):
        print_dialog()


In [ ]:
final_emo = None
t_emotion = None
t_prob = None
v_emotion = None
v_prob = None
sentence = None
response = None

while True:
    
    data_1 = sock.ReadReceivedData() # get trigger notification from unity
    time.sleep(1)
    
    if data_1 != None: # if NEW data has been received since last ReadReceivedData function call
        print(data_1) # print new received data
        #run models
        pipeline()
        #send emotion to unity
        sock.SendData(data_2) # Send this string to other application
        
        final_emo = None
        t_emotion = None
        t_prob = None
        v_emotion = None
        v_prob = None
        sentence = None
        response = None

### -------------------------------------------------------------------------------------

In [ ]:
# second way

In [ ]:
def send_data(final_emotion):
    print("script has started")
    # Create UDP socket to use for sending
    sock = U.UdpComms(udpIP="127.0.0.1", portTX=8000, portRX=8001, enableRX=True, suppressWarnings=True)
    sock.SendData('Sent from Python: ' + str(i)) # Send this string to other application
    print("!! Data has been sent !!")
    time.sleep(1)

def recieve_data():
    print("script has started")
    # Create UDP socket to use for receiving
    sock = U.UdpComms(udpIP="127.0.0.1", portTX=8000, portRX=8001, enableRX=True, suppressWarnings=True)
    while True:
        data_1 = sock.ReadReceivedData() # read data
        print("!! Trigger has been recieved !!")
        time.sleep(1)
        if data_1 != None: # if NEW data has been received since last ReadReceivedData function call
            break
    return true
    

In [ ]:
# final_emo = None
t_emotion = None
t_prob = None
v_emotion = None
v_prob = None
sentence = None
response = None

while(True):
    if(recieve_data()==True):
        
        t1 = Thread(target = speak_1)
        t2 = Thread(target = video_based_emotion)

        t1.start()
        t2.start()
        t1.join()
        t2.join()

        output_file = open(main_path + "outputs/text-output.txt", "r")
        sentence = output_file.read()
        output_file.close()

        responseGenerate(sentence)
        text_based_emotion(response)

        final_emo, final_prob = predict_emotion()
        synthesize(response)

        if(final_emo!=None):
            send_data(final_emotion)
            print_dialog()
            
            final_emo = None
            t_emotion = None
            t_prob = None
            v_emotion = None
            v_prob = None
            sentence = None
            response = None

In [ ]:
import simpleaudio as sa

filename = 'C:/Users/Munia/Documents/Anaconda/Master Thesis Project/vh-models/text to speech/generated_sound/audio_after.wav'
wave_obj = sa.WaveObject.from_wave_file(filename)
play_obj = wave_obj.play()
play_obj.wait_done()  # Wait until sound has finished playing